In [3]:
import networkx as nx
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import time

print("Loading P4_gentle_mound.qasm...")
qc = QuantumCircuit.from_qasm_file("P4_gentle_mound.qasm")
if 'measure' not in qc.count_ops():
    qc.measure_all()

print("Applying Graph Optimization...")
qubit_map = {q: i for i, q in enumerate(qc.qubits)}
G = nx.Graph()
G.add_nodes_from(range(qc.num_qubits))
for instruction in qc.data:
    if instruction.qubits and len(instruction.qubits) > 1:
        indices = [qubit_map[q] for q in instruction.qubits]
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                G.add_edge(indices[i], indices[j])

perm = list(nx.utils.reverse_cuthill_mckee_ordering(G))

qc_opt = transpile(qc, initial_layout=perm, optimization_level=3)

sim = AerSimulator(method='matrix_product_state', matrix_product_state_max_bond_dimension=128)

print("Running Corrected Simulation...")
job = sim.run(qc_opt, shots=4096)
counts = job.result().get_counts()

raw_winner = max(counts, key=counts.get)

standard_order = raw_winner[::-1]

print(f"Confidence: {(counts[raw_winner]/4096)*100:.2f}%")
print(f" Correct Flag (Standard):  {standard_order}")
print(f"   Raw Qiskit Output:        {raw_winner}")

Loading P4_gentle_mound.qasm...
Applying Graph Optimization...
Running Corrected Simulation...
Confidence: 6.57%
 Correct Flag (Standard):  0110101000010111001100100001010001101101
   Raw Qiskit Output:        1011011000101000010011001110100001010110
